# Dataton BC 2018

*Dirección de Capacidades Analíticas y Gobierno de Información, Grupo Bancolombia*<br>
Esta versión: *2018-10-17*<br>
Documentación tablas

## Naturaleza de los datos

Los datos entregados en este reto corresponden a transacciones realizadas por clientes persona del banco vía [PSE](https://www.pse.com.co/inicio). Estas transacciones, a diferencia de las transacciones realizadas vía POS, no cuentan con un código [MCC](https://en.wikipedia.org/wiki/Merchant_category_code) atado a la transacción, que permite conocer la categoría de comercio a la que pertence el establecimiento de comercio donde se realiza la transacción. Adicionalmente, muchas de estas transferencias por PSE corresponden a transferencias de pagos de servicios públicos, seguros, colegios, arrendamientos, y otros gastos que pueden ser denominados como gastos grandes. En el marco de un sistema de gestión de finanzas personales, poder categorizar adecuadamente estas transacciones que se realizan por PSE es de suma importancia para contar con una foto completa de la actividad de gastos de los clientes. Para este reto, los equipos participantes tendrán acceso a una muestra de transacciones PSE que corresponden a algo más de 300 mil clientes (persona), seleccionados de manera aleatoria. La tabla de transacciones cuenta con 11.8 millones de registros (uno para cada transacción), realizados entre septiembre de 2016 y octubre de 2018.

**NOTA** Los datos han pasado por un proceso relativamente simple de curación, pero se han dejado algunos ruidos en la calidad de éstos con el fin de que los equipos también lleven a cabo un proceso de inspección y limpieza.


## Categorización propuesta por el equipo de analítica de personas

En el Banco ya se han llevado a cabo esfuerzos por categorizar transacciones provenientes del canal POS (con tarjetas débito y crédito), lo cual ha incluído, entre otras cosas, una depuración y limpieza de los códigos MCC. A continuación mostramos, a manera de referencia, la categorización propuesta por el equipo.

1. Comida
2. Hogar
3. Cuidado personal
4. Entretenimiento
5. Educación
6. Transporte
7. Viajes
8. Ahorro
9. Pago de deudas
10. Ingresos
11. Retiros en efectivo
12. Mascotas
13. moda
14. Tecnología y comunicaciones
15. Otros

## NOTA IMPORTANTE

Recuerden que esta información aún contiene un elevado nivel de ruido. No solo no ha sido depurada de posibles datos atípicos (transacciones de valor muy elevado) fruto de errores o transacciones fallidas, sino que también cuenta con el ruido asociado al campo de referencia, donde se involucra el factor humano, ya que son campos de texto libre que pueden contener cualquier tipo de información.

Por seguridad, hemos eliminado cualquier número presente en dichos campos de referencia (cédulas, nits, montos, contratos, etc.).


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline
import matplotlib.animation as animation
import csv
matplotlib.style.use('seaborn')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import manifold
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
# from wordcloud import WordCloud

from mpl_toolkits.mplot3d import Axes3D

## Cargar el archivo de transacciones con marcas de tiempo

In [2]:
transacs_tot=pd.read_csv("transacs_tot_periods.csv",index_col=0)
transacs_tot.fec_hor=pd.to_datetime(transacs_tot.fec_hor)
print(transacs_tot.shape)
print(transacs_tot.dtypes)
transacs_tot.head()

/home/yagwar/anaconda3/envs/tensorflow/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(11816660, 17)
id_trn_ach                  int64
id_cliente                  int64
fec_hor            datetime64[ns]
valor_trx                 float64
ref                        object
sector                     object
subsector                  object
descripcion                object
dia_sem_num               float64
dia_mes_num               float64
dia_year_num              float64
hora_num                  float64
year_num                  float64
mes_num                   float64
semana_year_num           float64
period_mens                object
period_sem                 object
dtype: object


,id_trn_ach,id_cliente,fec_hor,valor_trx,ref,sector,subsector,descripcion,dia_sem_num,dia_mes_num,dia_year_num,hora_num,year_num,mes_num,semana_year_num,period_mens,period_sem
0,230435642,3,2016-12-07 11:34:51,2122392.51,CC,NaN,NaN,NaN,2.0,7.0,342.0,11.0,2016.0,12.0,49.0,2016-12,2016-12-05/2016-12-11
1,222356110,10,2016-10-16 03:42:04,148438.37,Referencia: Contrato: Valor: CC,NaN,NaN,NaN,6.0,16.0,290.0,3.0,2016.0,10.0,41.0,2016-10,2016-10-10/2016-10-16
2,309137749,10,2018-01-20 19:50:42,94025.19,CC,NaN,NaN,NaN,5.0,20.0,20.0,19.0,2018.0,1.0,3.0,2018-01,2018-01-15/2018-01-21
3,324614737,10,2018-03-26 19:21:46,94430.07,CC,NaN,NaN,NaN,0.0,26.0,85.0,19.0,2018.0,3.0,13.0,2018-03,2018-03-26/2018-04-01
4,235344690,18,2017-01-06 20:13:17,670645.57,MEDICINA PREPAGADA COLSANITAS CE,NaN,NaN,NaN,4.0,6.0,6.0,20.0,2017.0,1.0,1.0,2017-01,2017-01-02/2017-01-08


In [3]:
transac_pag=pd.read_csv("client_fr_val.csv",index_col=0)
transac_pag.head()

,id_cliente,count,valor_trx_suma
0,1,26,42769383.24
1,2,6,1365034.29
2,3,53,12058409.54
3,4,7,2534093.31
4,5,27,10990697.40


### conteo de frecuencias

In [4]:
transac_muest=transacs_tot[["id_trn_ach","id_cliente","valor_trx","dia_sem_num","dia_mes_num","hora_num","semana_year_num"]]
transac_muest.head()

,id_trn_ach,id_cliente,valor_trx,dia_sem_num,dia_mes_num,hora_num,semana_year_num
0,230435642,3,2122392.51,2.0,7.0,11.0,49.0
1,222356110,10,148438.37,6.0,16.0,3.0,41.0
2,309137749,10,94025.19,5.0,20.0,19.0,3.0
3,324614737,10,94430.07,0.0,26.0,19.0,13.0
4,235344690,18,670645.57,4.0,6.0,20.0,1.0


In [5]:
indic_analiz=columns=["dia_sem_num","dia_mes_num","hora_num","semana_year_num"]

In [6]:
client_dia_sem=pd.crosstab(transac_muest.id_cliente, columns=transac_muest["dia_sem_num"])
client_dia_sem.columns=["dia_sem_"+str(nm_col) for nm_col in client_dia_sem.columns]
client_dia_sem["id_cliente"]=client_dia_sem.index
transac_pag=pd.merge(transac_pag, client_dia_sem, on='id_cliente')

client_dia_mes=pd.crosstab(transac_muest.id_cliente, columns=transac_muest["dia_mes_num"])
client_dia_mes.columns=["dia_mes_"+str(nm_col) for nm_col in client_dia_mes.columns]
client_dia_mes["id_cliente"]=client_dia_mes.index
transac_pag=pd.merge(transac_pag, client_dia_mes, on='id_cliente')

client_hora_num=pd.crosstab(transac_muest.id_cliente, columns=transac_muest["hora_num"])
client_hora_num.columns=["hora_num_"+str(nm_col) for nm_col in client_hora_num.columns]
client_hora_num["id_cliente"]=client_hora_num.index
transac_pag=pd.merge(transac_pag, client_hora_num, on='id_cliente')

client_semana_year=pd.crosstab(transac_muest.id_cliente, columns=transac_muest["semana_year_num"])
client_semana_year.columns=["semana_year_"+str(nm_col) for nm_col in client_semana_year.columns]
client_semana_year["id_cliente"]=client_semana_year.index
transac_pag=pd.merge(transac_pag, client_semana_year, on='id_cliente')


### suma de totales

In [7]:
client_dia_sem_sum=pd.pivot_table(transac_muest,index=transac_muest.id_cliente,columns=transac_muest.dia_sem_num,values="valor_trx",aggfunc='sum')
client_dia_sem_sum.columns=["dia_sem_sum_"+str(nm_col) for nm_col in client_dia_sem_sum.columns]
client_dia_sem_sum["id_cliente"]=client_dia_sem_sum.index
transac_pag=pd.merge(transac_pag, client_dia_sem_sum, on='id_cliente')

client_dia_mes_sum=pd.pivot_table(transac_muest,index=transac_muest.id_cliente,columns=transac_muest.dia_mes_num,values="valor_trx",aggfunc='sum')
client_dia_mes_sum.columns=["dia_mes_sum_"+str(nm_col) for nm_col in client_dia_mes_sum.columns]
client_dia_mes_sum["id_cliente"]=client_dia_mes_sum.index
transac_pag=pd.merge(transac_pag, client_dia_mes_sum, on='id_cliente')

client_hora_num_sum=pd.pivot_table(transac_muest,index=transac_muest.id_cliente,columns=transac_muest.hora_num,values="valor_trx",aggfunc='sum')
client_hora_num_sum.columns=["hora_num_sum_"+str(nm_col) for nm_col in client_hora_num_sum.columns]
client_hora_num_sum["id_cliente"]=client_hora_num_sum.index
transac_pag=pd.merge(transac_pag, client_hora_num_sum, on='id_cliente')

client_semana_year_sum=pd.pivot_table(transac_muest,index=transac_muest.id_cliente,columns=transac_muest.semana_year_num,values="valor_trx",aggfunc='sum')
client_semana_year_sum.columns=["semana_year_sum_"+str(nm_col) for nm_col in client_semana_year_sum.columns]
client_semana_year_sum["id_cliente"]=client_semana_year_sum.index
transac_pag=pd.merge(transac_pag, client_semana_year_sum, on='id_cliente')


In [8]:
transac_pag.head()

,id_cliente,count,valor_trx_suma,dia_sem_0.0,dia_sem_1.0,dia_sem_2.0,dia_sem_3.0,dia_sem_4.0,dia_sem_5.0,dia_sem_6.0,...,semana_year_sum_44.0,semana_year_sum_45.0,semana_year_sum_46.0,semana_year_sum_47.0,semana_year_sum_48.0,semana_year_sum_49.0,semana_year_sum_50.0,semana_year_sum_51.0,semana_year_sum_52.0,semana_year_sum_53.0
0,1,26,42769383.24,11,6,6,2,1,0,0,...,NaN,1765195.73,NaN,79652.03,NaN,NaN,3395078.84,NaN,NaN,NaN
1,2,6,1365034.29,1,0,2,0,3,0,0,...,NaN,228017.43,NaN,NaN,NaN,NaN,227118.80,NaN,NaN,NaN
2,3,53,12058409.54,4,23,8,6,12,0,0,...,354125.88,362422.61,NaN,NaN,NaN,2376708.76,NaN,346696.94,NaN,NaN
3,4,7,2534093.31,0,0,1,2,4,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,27,10990697.40,3,6,7,6,5,0,0,...,825055.41,NaN,NaN,NaN,449752.12,388489.09,NaN,NaN,NaN,NaN


In [9]:
transac_pag.describe()

,id_cliente,count,valor_trx_suma,dia_sem_0.0,dia_sem_1.0,dia_sem_2.0,dia_sem_3.0,dia_sem_4.0,dia_sem_5.0,dia_sem_6.0,...,semana_year_sum_44.0,semana_year_sum_45.0,semana_year_sum_46.0,semana_year_sum_47.0,semana_year_sum_48.0,semana_year_sum_49.0,semana_year_sum_50.0,semana_year_sum_51.0,semana_year_sum_52.0,semana_year_sum_53.0
count,338416.000000,338416.000000,3.384160e+05,338416.000000,338416.000000,338416.000000,338416.000000,338416.000000,338416.000000,338416.000000,...,9.804300e+04,8.023100e+04,8.659600e+04,8.253900e+04,9.763000e+04,9.349700e+04,9.921000e+04,9.021000e+04,8.161900e+04,21.000000
mean,169297.714951,34.917551,1.225048e+07,5.872763,6.303588,6.175467,5.846281,5.810700,2.876900,2.029993,...,8.055379e+05,7.453952e+05,6.589276e+05,6.315915e+05,7.667661e+05,8.749162e+05,8.302448e+05,7.343012e+05,6.725359e+05,215621.043333
std,97744.394669,60.903217,3.502451e+07,10.977959,12.368396,11.936412,11.197788,11.259732,5.373782,3.854893,...,1.942496e+06,1.961277e+06,1.781120e+06,1.729009e+06,1.861113e+06,2.189195e+06,2.226831e+06,2.008766e+06,1.837719e+06,225068.630223
min,1.000000,1.000000,9.576800e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.541300e+02,9.523600e+02,9.525600e+02,9.532200e+02,9.503700e+02,9.515900e+02,9.552200e+02,9.516500e+02,9.504100e+02,4957.020000
25%,84652.750000,4.000000,5.655650e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.893027e+04,8.917321e+04,8.925854e+04,7.609808e+04,9.395977e+04,1.007880e+05,9.827208e+04,8.552115e+04,7.751160e+04,51458.610000
50%,169294.500000,16.000000,3.073673e+06,2.000000,3.000000,3.000000,2.000000,2.000000,1.000000,1.000000,...,2.833213e+05,2.475554e+05,2.257628e+05,2.045345e+05,2.599685e+05,2.862552e+05,2.622511e+05,2.215687e+05,2.071545e+05,88583.110000
75%,253943.250000,47.000000,1.185636e+07,8.000000,8.000000,8.000000,8.000000,8.000000,4.000000,2.000000,...,8.095193e+05,6.978017e+05,6.022598e+05,5.646677e+05,7.483576e+05,8.402825e+05,7.302906e+05,6.229825e+05,5.819138e+05,318434.700000
max,338606.000000,6185.000000,5.391350e+09,1075.000000,1293.000000,1179.000000,1114.000000,1116.000000,669.000000,212.000000,...,1.356906e+08,1.427394e+08,1.168153e+08,1.083783e+08,1.111531e+08,1.345009e+08,1.201460e+08,1.036490e+08,8.288006e+07,724401.620000


In [10]:
transac_pag.dtypes

id_cliente                int64
count                     int64
valor_trx_suma          float64
dia_sem_0.0               int64
dia_sem_1.0               int64
dia_sem_2.0               int64
dia_sem_3.0               int64
dia_sem_4.0               int64
dia_sem_5.0               int64
dia_sem_6.0               int64
dia_mes_1.0               int64
dia_mes_2.0               int64
dia_mes_3.0               int64
dia_mes_4.0               int64
dia_mes_5.0               int64
dia_mes_6.0               int64
dia_mes_7.0               int64
dia_mes_8.0               int64
dia_mes_9.0               int64
dia_mes_10.0              int64
dia_mes_11.0              int64
dia_mes_12.0              int64
dia_mes_13.0              int64
dia_mes_14.0              int64
dia_mes_15.0              int64
dia_mes_16.0              int64
dia_mes_17.0              int64
dia_mes_18.0              int64
dia_mes_19.0              int64
dia_mes_20.0              int64
                         ...   
semana_y

In [11]:
transac_pag.to_csv("client_fr_sum_periods.csv")